In [13]:
from scipy import stats
from scipy.stats import pearsonr, kendalltau

srcc = stats.spearmanr( [0.6480761, 0.62233955, 0.6661059, 0.631339, 0.6192408, 0.641552, 0.6423639, 0.7225227, 0.65772206, 0.63077545, 0.54655486  , 0.6763791, 0.62823856, 0.63159883, 0.62590545, 0.65006614, 0.6068853, 0.63621056, 0.5723472, 0.6442189, 0.62065214, 0.67560524, 0.6549503, 0.61165047], [0.062, 0.032, 0.029, 0.028, 0.027, 0.104, 0.028, 0.006, 0.033, 0.088, 0.052, 0.014, 0.029, 0.032, 0.022, 0.0, 0.012, 0.04, 0.051, 0.003, 0.032, 0.031, 0.019, 0.042])
srcc0 = pearsonr( [0.6480761, 0.62233955, 0.6661059, 0.631339, 0.6192408, 0.641552, 0.6423639, 0.7225227, 0.65772206, 0.63077545, 0.54655486  , 0.6763791, 0.62823856, 0.63159883, 0.62590545, 0.65006614, 0.6068853, 0.63621056, 0.5723472, 0.6442189, 0.62065214, 0.67560524, 0.6549503, 0.61165047], [0.062, 0.032, 0.029, 0.028, 0.027, 0.104, 0.028, 0.006, 0.033, 0.088, 0.052, 0.014, 0.029, 0.032, 0.022, 0.0, 0.012, 0.04, 0.051, 0.003, 0.032, 0.031, 0.019, 0.042])
srcc1 = kendalltau( [0.6480761, 0.62233955, 0.6661059, 0.631339, 0.6192408, 0.641552, 0.6423639, 0.7225227, 0.65772206, 0.63077545, 0.54655486  , 0.6763791, 0.62823856, 0.63159883, 0.62590545, 0.65006614, 0.6068853, 0.63621056, 0.5723472, 0.6442189, 0.62065214, 0.67560524, 0.6549503, 0.61165047], [0.062, 0.032, 0.029, 0.028, 0.027, 0.104, 0.028, 0.006, 0.033, 0.088, 0.052, 0.014, 0.029, 0.032, 0.022, 0.0, 0.012, 0.04, 0.051, 0.003, 0.032, 0.031, 0.019, 0.042])

srcc[0],srcc0[0],srcc1[0]

(-0.3369615411989142, -0.31079259594045033, -0.23766989273624328)

In [2]:
import torch
print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())


NVIDIA GeForce GTX 1050
True


In [3]:
%cd ..


d:\quality_assessment\knowledge_distillation\cvr-iqa\CVRKD-IQA


In [4]:
from test_DistillationIQA_single import DistillationIQASolver


obj_org = DistillationIQASolver(net_mode="org",student_address='./model_zoo/NAR_student_cross_dataset.pth')

normal mode...
>>>>>  ./model_zoo/NAR_student_cross_dataset.pth


In [5]:
obj_stackV1 = DistillationIQASolver(net_mode="stackingV1",student_address='./model_zoo/stacking_ep98_saved_student_model.pth')

stacking modeV1...
>>>>>  ./model_zoo/stacking_ep98_saved_student_model.pth


In [6]:
obj_stackV2 = DistillationIQASolver(net_mode="stackingV2",student_address='./model_zoo/stackingV2-ep70.pth')

stacking modeV2(combined)...
>>>>>  ./model_zoo/stackingV2-ep70.pth


In [7]:

obj_org.cvr_on_single_image("tst_imgs/i01_19_4.bmp" , "tst_imgs/i01_23_1.bmp"),obj_stackV1.cvr_on_single_image("tst_imgs/i01_19_4.bmp" , "tst_imgs/i01_23_1.bmp"),obj_stackV2.cvr_on_single_image("tst_imgs/i01_19_4.bmp" , "tst_imgs/i01_23_1.bmp")

(2.449223279953003, 2.6805977821350098, 2.5710458755493164)

# Correlation


# dst:

In [8]:
import random , glob

def get_random_HR_images():
    all_images = glob.glob('D:\quality_assessment\knowledge_distillation\cvr-iqa\CVRKD-IQA\dataset\DIV2K_ref/val_HR/*')
    return random.sample(all_images, 1)[0]


get_random_HR_images()

'D:\\quality_assessment\\knowledge_distillation\\cvr-iqa\\CVRKD-IQA\\dataset\\DIV2K_ref/val_HR\\0899.png'

all imgs


In [9]:
import cv2

SPAQ_ADDRESS = "E:\SPAQ"

len(glob.glob(SPAQ_ADDRESS+"/SPAQ/TestImage/*.jpg"))


11125

In [14]:
import pandas as pd

def read_excel_column(file_path, column_name):
    df = pd.read_excel(file_path)
    return df[column_name]

names = read_excel_column(SPAQ_ADDRESS+"/MOS and Image attribute scores.xlsx" , "Image name")[:50]
moses = read_excel_column(SPAQ_ADDRESS+"/MOS and Image attribute scores.xlsx" , "MOS")[:50]


names[15], moses[15]

('00016.jpg', 76.14)

In [23]:
from tqdm import tqdm
print("num of imgs:", len(names))
print("org...")
for j in range(3):
    preds = []
    for im in tqdm(names):
        try:
            rr = obj_org.cvr_on_single_image(SPAQ_ADDRESS+"/SPAQ/TestImage/"+im , get_random_HR_images())
            preds.append(rr)
        except:
            print(">>>>> ",im)
            preds.append(50)
    
    print("round "+str(j+1)+": ",stats.spearmanr( preds , moses)[0] , pearsonr(preds, moses)[0] , kendalltau(preds, moses)[0])





num of imgs: 50
org...


100%|██████████| 50/50 [00:30<00:00,  1.62it/s]


round 1:  0.1607241470846721 0.17548679743954732 0.11596570995350598


100%|██████████| 50/50 [00:28<00:00,  1.73it/s]


round 2:  0.1385387404658736 0.17250167786067064 0.1028991510855053


100%|██████████| 50/50 [00:29<00:00,  1.67it/s]

round 3:  0.12783020004164838 0.20629401430302896 0.09799919151000505


In [21]:
print("stackingV1...")
for j in range(3):
    preds = []
    for im in tqdm(names):
        try:
            rr = obj_stackV1.cvr_on_single_image(SPAQ_ADDRESS+"/SPAQ/TestImage/"+im , get_random_HR_images())
            preds.append(rr)
        except:
            print(">>>>> ",im)
            preds.append(50)
    
    print("round "+str(j+1)+": ",stats.spearmanr( preds , moses)[0] , pearsonr(preds, moses)[0] , kendalltau(preds, moses)[0])




stackingV1...


100%|██████████| 50/50 [00:30<00:00,  1.65it/s]


round 1:  0.33071622377416693 0.27370375627723154 0.2466312986335127


100%|██████████| 50/50 [00:32<00:00,  1.56it/s]


round 2:  0.2991668468292522 0.23849410257182002 0.2188648610390113


100%|██████████| 50/50 [00:30<00:00,  1.64it/s]

round 3:  0.3426732935752077 0.3102338728986237 0.27439773622801417


In [22]:




print("stackingV2...")
for j in range(3):
    preds = []
    for im in tqdm(names):
        try:
            rr = obj_stackV2.cvr_on_single_image(SPAQ_ADDRESS+"/SPAQ/TestImage/"+im , get_random_HR_images())
            preds.append(rr)
        except:
            print(">>>>> ",im)
            preds.append(50)
    
    print("round "+str(j+1)+": ",stats.spearmanr( preds , moses)[0] , pearsonr(preds, moses)[0] , kendalltau(preds, moses)[0])




stackingV2...


100%|██████████| 50/50 [00:31<00:00,  1.57it/s]


round 1:  0.3042570050578077 0.33771587216632015 0.2074316220295107


100%|██████████| 50/50 [00:31<00:00,  1.58it/s]


round 2:  0.30492929010686215 0.3522372235461141 0.21559822132201112


100%|██████████| 50/50 [00:31<00:00,  1.61it/s]

round 3:  0.48025162682814626 0.4893495733133367 0.3544304092945183
